In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
  #accessing the html page and storing it into a BS object
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

  #creating a list to append values
raw_data=[]
  #targeting the table within the page
table = soup.find('table', class_='wikitable sortable')

  #iterating through each row in the table and appending the fields into the list
for word in table.find_all('td'):
    value = word.text.replace('\n','')
    raw_data.append(value)
  
   #reshaping the list to a to 288 rows x 3 columns. Then creating a dataframe on the reshaped list
df_postcode = pd.DataFrame(np.reshape(raw_data,(288,3)),columns=['Postcode','Borough','Neighbourhood'])

    #deleting the rows, by index, of those records that contain a Borough value of Not assigned. 
df_postcode.drop((df_postcode[df_postcode['Borough']=='Not assigned'].index), inplace=True) 

In [3]:
combined=[] 
  #casting the post codes to a set() to remove duplicates, then casting back into a list to iterate using the for loop below
unique_postcodes = list(set(df_postcode['Postcode']))

  #iterating through the dataframe to combine the neighbourhoods based on unique postcode. This also will check for any not assigned neighbourhoods and assign them the borough value
for i in range(len(unique_postcodes)):
    #returns a dataframe containing only the records of the current postcode, [i], in the list of unique_postcodes
  groupedby_postcode = df_postcode[df_postcode['Postcode'] == unique_postcodes[i]]
  
    #checking for any "not assigned" neighbourhoods, and then hardcoding them to the borough name
  if groupedby_postcode['Neighbourhood'].any() == 'Not assigned':
    groupedby_postcode['Neighbourhood'] = groupedby_postcode['Borough']
    
    #setting delimited for the join function of the neighbourhoods
  temp_combine_neighbourhoods=', '
    #adding fields together into a new list:
      #first parameter = the current postcode string value
      #second parameter = the current borough value, by string
      #third parameter = the current neighbourhood values, joined by the delimiter above
  combined.append([unique_postcodes[i],list(set(groupedby_postcode['Borough']))[0],temp_combine_neighbourhoods.join(list(set(groupedby_postcode['Neighbourhood'])))])

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
   #reshaping the list to a to 103 rows x 3 columns. Then creating a dataframe on the reshaped list
df_combined = pd.DataFrame(np.reshape(combined,(103,3)),columns=['Postcode','Borough','Neighbourhood'])
print(np.shape(df_combined))
df_combined.head(12)


(103, 3)


,Postcode,Borough,Neighbourhood
0,M7R,Mississauga,Canada Post Gateway Processing Centre
1,M6B,North York,Glencairn
2,M2R,North York,Willowdale West
3,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
4,M1H,Scarborough,Cedarbrae
5,M8Y,Etobicoke,"Royal York South East, Kingsway Park South Eas..."
6,M4S,Central Toronto,Davisville
7,M3C,North York,"Flemingdon Park, Don Mills South"
8,M3N,North York,Downsview Northwest
9,M5S,Downtown Toronto,"Harbord, University of Toronto"
